In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create('es0')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [3]:
# 案例:查询苹果公司的产品
es.index('es0', id=0, body={"content": "Apple Mac"})
es.index('es0', id=1, body={"content": "Apple iPad"})
es.index('es0', id=2, body={"content": "Apple employee like Apple Pie and Apple juice"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [16]:
# 方式一:
es.search(index='es0', body={
    "query": {
        "bool": {
            "must":
                [
                    {
                        "match":
                            {
                                "content":
                                    {
                                        "query": "apple"
                                    }
                            }
                    }
                ],
            "must_not": [
                {
                    "match": {
                        "content": {
                            "query": "pie"
                        }
                    }
                }
            ]
        }
    }
})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.16786805,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.16786805,
    '_source': {'content': 'Apple Mac'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.16786805,
    '_source': {'content': 'Apple iPad'}}]}}

In [24]:
body = {"query": {
    "boosting": {
        # 单一查询条件({查询条件})
        # 
        # 多组查询条件([{查询条件1}, {查询条件2}, ...])
        # positive: 所有列出的查询条件都必须匹配
        # negative: 所有列出的查询条件都必须匹配
        "positive": [
            {
                "match": {
                    "content": {
                        "query": "apple"
                    }
                }
            }
        ],
        "negative": {
            "match": {
                "content": {
                    "query": "pie"
                }
            }
        },
        # negative_boost:
        # (Required, float) Floating point number between 0 and 1.0 used to decrease the relevance scores of documents matching the negative query.
        "negative_boost": 0.5
    }
}
}

# 方法二:
'''
GET es0/_search
`body`
'''
es.search(index='es0', body=body)

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 0.17280531,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 0.17280531,
    '_source': {'content': 'Apple employee like Apple Pie and Apple juice'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.16786805,
    '_source': {'content': 'Apple Mac'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.16786805,
    '_source': {'content': 'Apple iPad'}}]}}

In [33]:
es.indices.delete('es0')

{'acknowledged': True}